In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

ModuleNotFoundError: No module named 'tensorflow.examples'

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 50

# clears the memory of all variables left from previous run
# reset the computational graph, rather then start from the beginning
#tf.reset_default_graph()

# inputs -> 784x1
# targets -> 10x1
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

# weights_1 -> 784x1 * 50x1 -> 784x50
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
# biases_1 -> 1x50
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

# nn is a module of tf and contains neural network support
# among other things, it contains the most commonly used activation functions
# tanh, relu, softmax, sigmoid
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 =  tf.nn.relu(tf.matmul(outputs_1,weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])

# biases_3 -> Nonexoutput_size 
biases_3 = tf.get_variable("biases_3", [output_size])

# we use a trick at the output layer and don't use activation function as relu
outputs = tf.matmul(outputs_2, weights_3) + biases_3

# applies a softmax activation and calculates a cross-entropy loss
# this function scales our output_layer, even if it has a too small numbers, at the end they'll all be scaled
    # and problem will be solved
loss = tf.nn.sotmax_cross_entropy_with_logits(logits=outputs, labels=targets)

# finds the mean of the elements of a tensor across a dimensions
mean_loss = tf.reduce_mean(loss) 

# try to minimize mean_loss with optimizer
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

# compare the all indices one by one and store the output as boolean 1 or 0 to the out_equals_target columns vector
# out_equals_target is a column vector that contains just 0s and 1s (not equal or equal)
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

# tf.cast is a method that cast an object to another data type
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

batch_size = 100

batches_number = mnist.train._num_examples // batch_size

max_epochs = 15

prev_validation_loss = 9999999.

for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0.
    
    for batch_counter in range (batches_numer):
        
        _, batch_loss = sess.run([optimize, mean_loss],
                                 feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= batches_number
    
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                                    feed_dict={inputs: input_batch, targets: target_batch})
    
    print('Epoch' + str(epoch_counter+1)+
          '.Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy*100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
    
    prev_validation_loss = validation_loss

print('Validation loss is higher then the previous validation loss. Session has ended.')
    

In [ ]:
# you can play with the hyperparameters and get better results
    # hidden layer number (how many hidden layer should we use? 1,2 or more...) -> Width
    # hidden_layer_size (50x1 or 100x1 or maybe even more rows?) -> Depth
    # learning_rate (lower is slower, higher can miss the the global min or can oscillate)
    # batch_size (batch_size = 1 -> SGD, that is slower. Higher the batch_size -> lower the acc but faster)
    # activations (can give better or worse result. Depends on the situation.)

In [ ]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy],
                         feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0]*100.

print('Test acc: '+'{0:2f}'.format(test_accuracy_percent)+'%')